In [1]:
import math

import torch

from script.NeuralNets.Networks import SequentialNN, ICNN, ICNNApproxMax, ICNNLogical
from script.settings import device, data_type
import script.DHOV.MultiDHOV as multidhov
from script.Verification.Verifier import SingleNeuronVerifier, MILPVerifier, DHOVVerifier
import gurobipy as grp
from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms import Compose, ToTensor, Normalize
from script.NeuralNets.ICNNFactory import ICNNFactory

In [36]:
def add_max_constr(model, neuron_name):
    neuron_var = model.getVarByName(neuron_name)
    model.setObjective(neuron_var, grp.GRB.MAXIMIZE)

def add_min_constr(model, neuron):
    neuron_var = model.getVarByName(neuron)
    model.setObjective(neuron_var, grp.GRB.MINIMIZE)

In [3]:
def optimize_model(model, neuron_name):
    model.update()
    model.optimize()
    if model.Status == grp.GRB.OPTIMAL:
        print("opt value: {}".format(model.getVarByName(neuron_name).getAttr("x")))

In [4]:
def icnn_model(icnn, nn, input_x, eps, layer_index, from_neuron, to_neuron, print_log=False):
    m = grp.Model()
    if not print_log:
        m.Params.LogToConsole = 0

    input_flattened = torch.flatten(input_x)
    bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds(
        [input_flattened.add(-eps), input_flattened.add(eps)])

    parameter_list = list(nn.parameters())

    input_size = len(parameter_list[2*(layer_index-1)])
    lb = bounds_layer_out[layer_index-1][0].detach().cpu().numpy()
    ub = bounds_layer_out[layer_index-1][1].detach().cpu().numpy()
    in_var = m.addMVar(input_size, lb=-float("inf"), ub=float("inf"), name="icnn_var")

    low = bounds_layer_out[layer_index - 1][0][from_neuron: to_neuron]
    up = bounds_layer_out[layer_index - 1][1][from_neuron: to_neuron]
    low = torch.zeros_like(low, dtype=data_type).to(device) - 1000
    up = torch.zeros_like(low, dtype=data_type).to(device) + 1000
    constraint_bounds_affine_out, constraint_bounds_layer_out = icnn.calculate_box_bounds([low, up])
    icnn.add_max_output_constraints(m, in_var[from_neuron: to_neuron], constraint_bounds_affine_out, constraint_bounds_layer_out)

    return m

In [5]:
"""W1 = [1. 1.; 1. -1.]
    b1 = [0., 0.]
    W2 = [1. 1.; 1. -1.]
    b2 = [-0.5, 0.]
    W3 = [-1. 1.; 1. 1.]
    b3 = [3., 0.] """

"""nn = SequentialNN([2, 2, 2, 2])

with torch.no_grad():
    parameter_list = list(nn.parameters())
    parameter_list[0].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[1].data = torch.tensor([0, 0], dtype=data_type).to(device)
    parameter_list[2].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[3].data = torch.tensor([-0.5, 0], dtype=data_type).to(device)
    parameter_list[4].data = torch.tensor([[-1, 1], [1, 1]], dtype=data_type).to(device)
    parameter_list[5].data = torch.tensor([3, 0], dtype=data_type).to(device)

test_image = torch.tensor([[0, 0]], dtype=data_type).to(device)"""

"""transform = Compose([ToTensor(),
                         Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                        )

training_data = CIFAR10(root="../../cifar", train=True, download=True, transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([32 * 32 * 3, 1024, 512, 10])
nn.load_state_dict(torch.load("../../cifar_fc.pth", map_location=torch.device(device)), strict=False)
"""

transform = Compose([ToTensor(),
                         Normalize(0.5, 0.5)]
                        )

training_data = MNIST(root="../../mnist",
                      train=True,
                      download=True,
                      transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([28*28*1, 100, 30, 10])
nn.load_state_dict(torch.load("../../mnist_fc.pth", map_location=torch.device('cpu')), strict=False)

parameter_list = list(nn.parameters())

"""nn = SequentialNN([50, 50, 50, 7])
test_image = torch.zeros((1, 50), dtype=data_type).to(device)
parameter_list = list(nn.parameters())"""

'nn = SequentialNN([50, 50, 50, 7])\ntest_image = torch.zeros((1, 50), dtype=data_type).to(device)\nparameter_list = list(nn.parameters())'

In [ ]:
eps = 0.01

Test for DHOV

In [7]:
group_size = 2
icnn_factory = ICNNFactory("logical", [5, 5, 1], force_positive_init=False, with_two_layers=False,
                               init_scaling=10, init_all_with_zeros=False)
#icnn_factory = ICNNFactory("standard", [5, 5, 1])

dhov_verifier = multidhov.MultiDHOV()
dhov_verifier.start_verification(nn, test_image, icnn_factory, group_size, eps=eps, icnn_epochs=10,
                                 icnn_batch_size=1000, sample_count=1000, sample_new=True, use_over_approximation=True, break_after=None,
                                 sample_over_input_space=False, sample_over_output_space=True, use_icnn_bounds=True,
                                 use_fixed_neurons=True, sampling_method="per_group_sampling",
                                 force_inclusion_steps=0, preemptive_stop=False, even_gradient_training=False,
                                 keep_ambient_space=True, data_grad_descent_steps=0, opt_steps_gd=100,
                                 train_outer=False, print_training_loss=False,
                                 should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="none")


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-12


C:\Users\Ufuk\Documents\Programming\ICNN_verification\script\DHOV\MultiDHOV.py:77: UserWarning: keep_ambient_space is True and sampling method ist -per_group_sampling-. Keeping previous samples is not supported when using per group sampling
  warnings.warn("keep_ambient_space is True and sampling method ist -per_group_sampling-. "



approximation of layer: 0
    number of fixed neurons for current layer: 92
    layer progress, group 1 of 4 
        time for sampling for one group: 0.008049249649047852


C:\Users\Ufuk\Documents\Programming\ICNN_verification\script\Optimizer\sdlbfgs.py:83: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ..\torch\csrc\utils\python_arg_parser.cpp:1420.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


        time for training: 0.33118152618408203
        actual verification time 0.022214889526367188
        time for verification: 0.1030421257019043
    layer progress, group 2 of 4 
        time for sampling for one group: 0.010093450546264648
        time for training: 0.36333727836608887
        actual verification time 0.020172119140625
        time for verification: 0.10095024108886719
    layer progress, group 3 of 4 
        time for sampling for one group: 0.010094404220581055
        time for training: 0.4041421413421631
        actual verification time 0.020164012908935547
        time for verification: 0.10088872909545898
    layer progress, group 4 of 4 
        time for sampling for one group: 0.018135786056518555
        time for training: 0.41382408142089844
        actual verification time 0.03026556968688965
        time for verification: 0.09084820747375488
    time for regrouping method: 0.0

approximation of layer: 1
        lower: new -0.07899962427050938, old -1

In [39]:
layer_index = 2
neuron_index = 1
# neuron_name = "relu_var{}[{}]".format(2*layer_index, neuron_index)
neuron_name = "last_affine_var[{}]".format(neuron_index)

In [40]:
print(dhov_verifier.all_group_indices)

[[[1, 23], [25, 51], [60, 69], [86, 96]], [[0]]]


In [41]:
print(dhov_verifier.bounds_affine_out[layer_index][0])

tensor([-2.1587e-04,  7.3969e-01, -5.7569e-01,  8.4478e+00, -1.3536e+01,
         9.8993e+00, -4.1025e+00,  1.4756e+00, -4.4561e+00, -3.1324e+00],
       dtype=torch.float64, grad_fn=<CopySlices>)


In [42]:
print(dhov_verifier.bounds_affine_out[layer_index][1])

tensor([  1.1198,   2.0933,   1.4066,  10.0364, -11.8492,  11.4841,  -2.8242,
          2.8489,  -2.9585,  -1.6241], dtype=torch.float64,
       grad_fn=<CopySlices>)


In [43]:
input_flattened = torch.flatten(test_image)
simple_bounds_affine_out, simple_bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])
print(simple_bounds_affine_out[layer_index][0])
print(simple_bounds_affine_out[layer_index][1])

tensor([ -5.2223,  -4.9188,  -6.6589,   2.6449, -18.6182,   3.6405,  -9.5669,
         -4.3524, -10.9687,  -9.1357], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([ 6.4550,  7.2957,  7.8522, 15.6069, -5.7168, 17.2791,  2.3780,  8.4534,
         3.7083,  4.3367], dtype=torch.float64, grad_fn=<AddBackward0>)


In [44]:
icnn_neuron_name = "output_layer_[{}]_[{}]".format(layer_index, neuron_index)

In [45]:
dhov_copy = dhov_verifier.nn_encoding_model.copy()
add_min_constr(dhov_copy, icnn_neuron_name)
dhov_copy.update()
optimize_model(dhov_copy, icnn_neuron_name)

opt value: 0.7396900527388428


In [46]:
dhov_copy = dhov_verifier.nn_encoding_model.copy()
dhov_copy.Params.LogToConsole = 0
add_max_constr(dhov_copy, icnn_neuron_name)
optimize_model(dhov_copy, icnn_neuron_name)

opt value: 2.0932818295190487


Test for SNV

In [47]:
snv_verifier = SingleNeuronVerifier(nn, test_image, eps, print_log=False)
snv_verifier.generate_constraints_for_net()
snv_model = snv_verifier.model
snv_model.update()

In [48]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_min_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 0.5887688158971365


In [49]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_max_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 2.0317086474039625


Test for MILP

In [50]:
milp_verifier = MILPVerifier(nn, test_image, eps, print_log=False)
milp_verifier.generate_constraints_for_net(until_layer_neuron=[layer_index, neuron_index])
milp_model = milp_verifier.model
milp_model.update()

"""all_var = milp_model.getVars()
for var in all_var:
    print(var)"""


'all_var = milp_model.getVars()\nfor var in all_var:\n    print(var)'

In [51]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_min_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.9777576260166984


In [52]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_max_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 1.8828109205406829
